## Creating a DB with land plots data

In [1]:
import glob
import re

import duckdb

In [2]:
con = duckdb.connect("plots_poland.duckdb")

In [3]:
con.execute("INSTALL spatial")
con.execute("LOAD spatial")

I decided to create a DB with a set of views. This will allow me to:  
✅ define the content of the DB only once  
✅ save disc space - the data itself stays in parquet/GPKG  

In [4]:
con.execute('''
CREATE VIEW plots AS 
SELECT * FROM read_parquet('input_data/woj_mazowieckie/plots.parquet')
''')

In [5]:
con.execute('''
    CREATE VIEW transactions AS 
    SELECT * FROM read_parquet('input_data/woj_mazowieckie/transactions.parquet')
''')

The `plots.parquet` file contains all plots in the Masovian Voivodeiship - geometry with a few attributes. The `transactions.parquet` file allows to see which plots were sold/bought in the last years. 

In [6]:
BDOT_DATA_FOLDER = 'input_data/woj_mazowieckie/bdot10k/'

In [7]:
bdot10k_paths = glob.glob(BDOT_DATA_FOLDER + '*.gpkg')
bdot10k_paths[:5]

['input_data/woj_mazowieckie/bdot10k\\PL.PZGiK.330.BDOT10k.14_OT_ADJA_A.gpkg',
 'input_data/woj_mazowieckie/bdot10k\\PL.PZGiK.330.BDOT10k.14_OT_ADMS_A.gpkg',
 'input_data/woj_mazowieckie/bdot10k\\PL.PZGiK.330.BDOT10k.14_OT_ADMS_P.gpkg',
 'input_data/woj_mazowieckie/bdot10k\\PL.PZGiK.330.BDOT10k.14_OT_BUBD_A.gpkg',
 'input_data/woj_mazowieckie/bdot10k\\PL.PZGiK.330.BDOT10k.14_OT_BUHD_A.gpkg']

In [8]:
bdot_identification = []
for p in bdot10k_paths:
    p_split = re.split(r'[._]', p)
    bdot_name = p_split[-3] + '_' + p_split[-2]
    bdot_identification.append((bdot_name, p))

In [9]:
for name, path in bdot_identification:
    con.execute(f"""
        CREATE VIEW {name} AS
        SELECT * FROM ST_Read('{path}')
    """)

For futher analyses, I also added BDOT10k - topographical objects data to the DB. Data from each BDOT10k layer were added to the DB as separate views. 

In [10]:
con.close()